In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
traval_tra_features = np.load('traval_tra_features.npy').astype(np.float32)
traval_tra_targets = np.load('traval_tra_targets.npy').astype(np.int64)[:,0]
traval_val_features = np.load('traval_val_features.npy').astype(np.float32)
traval_val_targets = np.load('traval_val_targets.npy').astype(np.int64)[:,0]
traval_tra_features.shape, traval_val_features.shape

((687636, 2600), (76405, 2600))

In [2]:
import torch
from torch import optim
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
import sklearn
from skorch.callbacks import EpochScoring, LRScheduler, Checkpoint
from torch.optim import Adam, SGD
import adamod

/home/amax/miniconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from skorch.dataset import Dataset
from skorch.helper import predefined_split
valid_ds = Dataset(traval_val_features, traval_val_targets)

In [4]:
class DownSample2x(nn.Sequential):
    def __init__(self, _in, _out):
        super().__init__(
            nn.Conv1d(_in, _out, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
        )

class SELayer(nn.Module):
    def __init__(self, _in, _hidden=64):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(_in, _hidden),
            nn.PReLU(),
            nn.Linear(_hidden, _in),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y
    
class ResConv1d(nn.Module):
    def __init__(self, _in, _out):
        super(ResConv1d, self).__init__()
        
        self.cal = nn.Sequential(
            nn.Conv1d(_in, _out, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm1d(_out),
            nn.ReLU(),
            nn.Conv1d(_out, _out, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm1d(_out),
        )
        self.se = SELayer(_out, _out)
        self.conv = nn.Conv1d(_in, _out, kernel_size=1, padding=0, stride=1)
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm1d(_out)
        
    def forward(self, x):
        res = self.cal(x)
        res = self.se(res)
        
        x = self.bn(self.conv(x))
        
        return self.relu(res + x)
        
class MyModule(nn.Module):
    def __init__(self, nonlin=F.relu):
        super(MyModule, self).__init__()
        
        self.d1 = DownSample2x(1, 64)
        self.c1 = ResConv1d(64, 64)
        
        self.d2 = DownSample2x(64, 64)
        self.c2 = ResConv1d(64, 64)
        
        self.d3 = DownSample2x(64, 64)
        self.c3 = ResConv1d(64, 64)
        
        self.d4 = DownSample2x(64, 64)
        self.c4 = ResConv1d(64, 64)
        
        self.d5 = DownSample2x(64, 64)
        self.c5 = ResConv1d(64, 64)
        
        self.d6 = DownSample2x(64, 64)
        self.c6 = ResConv1d(64, 64)
        
        self.dropout = nn.Dropout(0.5)
        self.cls = nn.Linear(64 * 40, 3)
        
    def preprocess(self, x, p=2, eps=1e-8):
        x = x / (x.norm(p=p, dim=1, keepdim=True)+eps)
        x = x.unsqueeze(1)
        return x
        
    def forward(self, x):
        bs = x.size(0)
        x = self.preprocess(x)
        
        x = self.d1(x)
        x = self.c1(x)
        
        x = self.d2(x)
        x = self.c2(x)
        
        x = self.d3(x)
        x = self.c3(x)

        x = self.d4(x)
        x = self.c4(x)
        
        x = self.d5(x)
        x = self.c5(x)
        
        x = self.d6(x)
        x = self.c6(x)
        
        x = x.reshape(bs, -1)
        x = self.dropout(x)

        return F.softmax(self.cls(x))

    
def microf1(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)
    return sklearn.metrics.f1_score(y_true, y_pred,average='micro')
def macrof1(net, ds, y=None):
    y_true = [y for _, y in ds]
    y_pred = net.predict(ds)
    return sklearn.metrics.f1_score(y_true, y_pred,average='macro')

class F1Loss(nn.Module):
    def __init__(self):
        super(F1Loss, self).__init__()

    def forward(self, input, target, eps=1e-10):
        loss = 0
        for idx, i in enumerate(torch.eye(3).cuda()):
            t = i.view(3,1)
            y_pred_ = input.matmul(t).squeeze()
            y_true_ = target==idx
            loss += 0.5 * (y_true_ * y_pred_).sum() / (y_true_ + y_pred_ + eps).sum()
        return -torch.log(loss+eps)

## General Training

In [5]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model1')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9666     0.9894        0.0340       0.9894        0.0356     +  310.5964
      2     0.9655     0.9891        0.0203       0.9891        0.0343        311.6362
      3     0.9789     0.9944        0.0176       0.9944        0.0190     +  309.7627
      4     0.9519     0.9881        0.0161       0.9881        0.0366        310.7354
      5     0.9790     0.9949        0.0151       0.9949        0.0163     +  309.8077
      6     0.9804     0.9950        0.0142       0.9950        0.0157     +  308.3822
      7     0.9805     0.9950        0.0138       0.9950        0.0161     +  309.8312
      8     0.9837     0.9960        0.0131       0.9960        0.0137     +  311.1483
      9     0.9831     0.9956        0.0126       0.9956        0.0149        309.0915
     10     0.9821     0.9956        0.0121

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

## MacroLoss Fintuning

In [7]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model1', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model1/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9844     0.9962        0.3072       0.9962        0.3091     +  313.8841
      2     0.9839     0.9960        0.3046       0.9960        0.3094        314.0102
      3     0.9847     0.9962        0.3039       0.9962        0.3090     +  312.0275
      4     0.9844     0.9962        0.3041       0.9962        0.3092        312.9366
      5     0.9846     0.9962        0.3031       0.9962        0.3088        313.3549
      6     0.9847     0.9962        0.3020       0.9962        0.3086     +  313.3883
      7     0.9842     0.9961        0.3034       0.9961        0.3096        313.3217
      8     0.9844     0.9962        0.3018       0.9962        0.3092        312.9318
      9     0.9837     0.9961        0.3021       0.9961        0.3096        309.0600
     10     0.9835     0.9961        0.3003

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [8]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model2')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model2', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model2/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9430     0.9858        0.0340       0.9858        0.0431     +  304.3791
      2     0.9772     0.9938        0.0204       0.9938        0.0205     +  303.8527
      3     0.9751     0.9931        0.0176       0.9931        0.0216        305.1009
      4     0.9770     0.9941        0.0160       0.9941        0.0194        303.8466
      5     0.9792     0.9948        0.0150       0.9948        0.0163     +  300.3223
      6     0.9832     0.9958        0.0143       0.9958        0.0145     +  302.7430
      7     0.9830     0.9957        0.0136       0.9957        0.0149        302.2970
      8     0.9833     0.9957        0.0130       0.9957        0.0140     +  303.8143
      9     0.9824     0.9956        0.0126       0.9956        0.0152        306.9870
     10     0.9834     0.9959        0.0121

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [9]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model3')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model3', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model3/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.0797     0.1245        0.0345       0.1245        8.5422     +  308.1514
      2     0.9768     0.9937        0.0203       0.9937        0.0214     +  305.9702
      3     0.9777     0.9945        0.0176       0.9945        0.0177     +  303.4543
      4     0.9804     0.9951        0.0164       0.9951        0.0164     +  304.3938
      5     0.9807     0.9951        0.0151       0.9951        0.0162     +  303.5369
      6     0.9801     0.9948        0.0142       0.9948        0.0180        306.5462
      7     0.9834     0.9958        0.0137       0.9958        0.0145     +  307.2421
      8     0.9824     0.9956        0.0130       0.9956        0.0155        306.8863
      9     0.9837     0.9958        0.0126       0.9958        0.0140     +  304.9817
     10     0.9782     0.9947        0.0122

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 

In [10]:
net = NeuralNetClassifier(
    MyModule,
    max_epochs=16,
    lr=0.001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model4')],
    device='cuda'
)

net.fit(traval_tra_features, traval_tra_targets)

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.0001,
    batch_size=128,
    optimizer=Adam,
    iterator_train__shuffle=True,
    criterion=F1Loss,
    train_split=predefined_split(valid_ds),
    callbacks=[EpochScoring(macrof1, use_caching=True, lower_is_better=False),
               EpochScoring(microf1, use_caching=True, lower_is_better=False),
               Checkpoint(monitor='macrof1_best', dirname='03_model4', fn_prefix='f1')],
    device='cuda'
)

net.initialize() 
net.load_params(f_params='03_model4/params.pt')
net.partial_fit(traval_tra_features, traval_tra_targets)

/home/amax/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:104: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    macrof1    microf1    train_loss    valid_acc    valid_loss    cp       dur
-------  ---------  ---------  ------------  -----------  ------------  ----  --------
      1     0.9670     0.9916        0.0342       0.9916        0.0258     +  303.3747
      2     0.9746     0.9936        0.0208       0.9936        0.0211     +  302.1407
      3     0.9798     0.9948        0.0178       0.9948        0.0176     +  305.9477
      4     0.9811     0.9953        0.0163       0.9953        0.0154     +  307.9575
      5     0.9809     0.9953        0.0150       0.9953        0.0150        307.7735
      6     0.9808     0.9952        0.0145       0.9952        0.0153        304.0878
      7     0.9795     0.9951        0.0137       0.9951        0.0162        303.3571
      8     0.9816     0.9956        0.0130       0.9956        0.0150     +  303.7928
      9     0.9826     0.9956        0.0126       0.9956        0.0155     +  304.1707
     10     0.9808     0.9953        0.0120

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=MyModule(
    (d1): DownSample2x(
      (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
      (1): ReLU(inplace=True)
    )
    (c1): ResConv1d(
      (cal): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
        (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool1d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=64, bias=True)
          (1): PReLU(num_parameters=1)
          (2): Linear(in_features=64, out_features=64, bias=True)
          (3): Sigmoid()
        )
      )
      (conv): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
      (relu): ReLU()
 